### 1. Import the necessary Libraries

In [22]:
import pandas as pd
import pathlib
import copy

### 2. Import the Dataset & convert the date info into compatible format (Python Datetime format)

In [23]:
path= str("measurements.csv")
data= pd.read_csv(path)
data['MSMT_DATE']=pd.to_datetime(data['MSMT_DATE'])
col=list(data.columns)

In [24]:
data

,SITE_CODE,WLM_ID,MSMT_DATE,WLM_RPE,WLM_GSE,GWE,GSE_GWE,WLM_QA_DESC,WLM_DESC,WLM_ACC_DESC,WLM_ORG_NAME,COOP_ORG_NAME,MONITORING_PROGRAM,MSMT_CMT
0,320000N1140000W001,3016758,2021-08-26 20:00:00,545.92,545.92,133.92,412.0,NaN,Electric sounder measurement,Water level accuracy to nearest hundredth of a...,Monterey Peninsula Water Management District,Monterey Peninsula Water Management District,VOLUNTARY,NaN
1,320000N1140000W001,3016757,2021-07-29 19:00:00,545.92,545.92,135.92,410.0,NaN,Electric sounder measurement,Water level accuracy to nearest hundredth of a...,Monterey Peninsula Water Management District,Monterey Peninsula Water Management District,VOLUNTARY,NaN
2,320000N1140000W001,3016756,2021-06-24 18:00:00,545.92,545.92,134.92,411.0,NaN,Electric sounder measurement,Water level accuracy to nearest hundredth of a...,Monterey Peninsula Water Management District,Monterey Peninsula Water Management District,VOLUNTARY,NaN
3,320000N1140000W001,3016755,2021-05-27 17:00:00,545.92,545.92,134.92,411.0,NaN,Electric sounder measurement,Water level accuracy to nearest hundredth of a...,Monterey Peninsula Water Management District,Monterey Peninsula Water Management District,VOLUNTARY,NaN
4,320000N1140000W001,3016754,2021-04-29 16:00:00,545.92,545.92,136.92,409.0,NaN,Electric sounder measurement,Water level accuracy to nearest hundredth of a...,Monterey Peninsula Water Management District,Monterey Peninsula Water Management District,VOLUNTARY,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2166650,420171N1214111W001,675330,2002-01-03 00:00:00,4061.00,4060.00,4016.30,43.7,NaN,Unknown,Water level accuracy is unknown,Department of Water Resources,Department of Water Resources,VOLUNTARY,NaN
2166651,420171N1214111W001,675329,2001-12-19 00:00:00,4061.00,4060.00,4014.90,45.1,NaN,Unknown,Water level accuracy is unknown,Department of Water Resources,Department of Water Resources,VOLUNTARY,NaN
2166652,420171N1214111W001,675328,2001-12-06 00:00:00,4061.00,4060.00,4011.20,48.8,NaN,Unknown,Water level accuracy is unknown,Department of Water Resources,Department of Water Resources,VOLUNTARY,NaN
2166653,420171N1214111W001,675327,2001-11-20 00:00:00,4061.00,4060.00,4012.60,47.4,NaN,Unknown,Water level accuracy is unknown,Department of Water Resources,Department of Water Resources,VOLUNTARY,NaN


### 3. Wrange the data to the proper format

In [25]:
data=data.drop(columns=['WLM_ID','WLM_RPE','WLM_GSE','GWE','WLM_QA_DESC','WLM_DESC','WLM_ACC_DESC','WLM_ORG_NAME','COOP_ORG_NAME','MONITORING_PROGRAM','MSMT_CMT']) #the name and Unnamed are both redundant so the are removed from the dataset

### 4. Create a Dictionary object using the wells names as Key and a list of all the readings as items

In [26]:
wellsDic={}
for c in range(len(data.index)):
    if data.iloc[c].SITE_CODE not in list(wellsDic.keys()):
        wellsDic[str(data.iloc[c].SITE_CODE)]=[data.iloc[c].tolist()]
    else:
        wellsDic[str(data.iloc[c].SITE_CODE)].append(data.iloc[c].tolist())

### 5. Take the Average Monthly reading for each well and add all the readings into a Dictunary Object using the wells names as key and the readings as items

In [27]:
v=list(wellsDic.keys())
for x in v:
    z=pd.DataFrame(wellsDic[x])
    wellsDic[x]=z.resample('M', on=1).mean()  #assgin the datatime column as index and resample the data on average monthly basis in it
    wellsDic[x]=wellsDic[x].reset_index()     #assign a new index and return the date column as a column to the dataset  

### 6. Remove the Wells that have low readings

#### 6.1 Check the number BEFORE the removal

In [28]:
print(len(wellsDic))

42255


In [29]:
wellsBckup=copy.deepcopy(wellsDic)

In [61]:
wellsDic=copy.deepcopy(wellsBckup)

#### 6.2 Remove the Wells

In [62]:
wells= list(data.SITE_CODE.unique()) #get the number of wells in the dataset
for x in wells:
    wellsDic[x].dropna(inplace=True)
    if len(wellsDic[x])<120: #Determine the threshold for the cutoff number of readings, in this case its (6)
        wellsDic.pop(x)

#### 6.3 Check the number AFTER the removal

### 7. Convert the Dictunary Object into DataFrame Object in preparation to export it to excel or CSV file

In [63]:
print(len(wellsDic))

2992


In [64]:
test=pd.DataFrame([0])
for x in list(wellsDic.keys()):
    rng = list(range(len(wellsDic[x].index)))
    for i in rng:
        test=test.append([[x,wellsDic[x].iloc[i,0],wellsDic[x].iloc[i,1]]], ignore_index=True)
test.rename(columns={0: 'Well Code',1: 'Date', 2: 'Readings'}, inplace=True)

In [65]:
test.nunique()

Well Code     2993
Date          1392
Readings     67013
dtype: int64

### 8. Export the Data to a new CSV file

In [66]:
test.to_csv('USData(10Y-2992).csv')